In [ ]:
import os
import json
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib import colors

DATE_STR = dt.datetime.now().strftime("%Y-%m-%d")
#DATE_STR = '2026-01-27'
APP_NAME = "recomart"

BASE_RAW = "storage\\raw"
BASE_PREPARED = "prepared"
BASE_LOGS = "logs"
BASE_PLOTS = "plots"
REPORT_PATH = "Raw_Data_Quality_Report.pdf"

LOG_FILE_PATH = os.path.join(BASE_LOGS, f"{APP_NAME}_{DATE_STR}.log")

def log_step(task, status, message):
    timestamp = dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    os.makedirs(BASE_LOGS, exist_ok=True)
    entry = f"[{timestamp}] [{task}] [{status}] {message}\n"
    with open(LOG_FILE_PATH, "a", encoding="utf-8") as f:
        f.write(entry)
    print(entry.strip())


for folder in [BASE_LOGS, BASE_PLOTS]:
    os.makedirs(folder, exist_ok=True)

In [15]:
PATHS = {
    "users": os.path.join(BASE_RAW, "users", DATE_STR),
    "products": os.path.join(BASE_RAW, "products", DATE_STR),
    "transactions": os.path.join(BASE_RAW, "transactions", DATE_STR)
}

log_step("TASK_4", "START", "Initiating data profiling and validation for data quality.")

try:
    log_step("TASK_4", "FETCH", f"Reading the first batch of user records from users1.csv.")
    users_raw = pd.read_parquet(os.path.join(PATHS["users"], "users_merged.parquet"))
    
    # log_step("TASK_4", "FETCH", f"Fetching the remaining user profiles from users2.csv.")
    # u2 = pd.read_parquet(os.path.join(PATHS["users"], "users2.csv"))
    # users_raw = pd.concat([u1, u2])
    
    log_step("TASK_4", "FETCH", "Opening the products.json file to extract item metadata.")
    prods = pd.read_parquet(os.path.join(PATHS["products"], "products.parquet"))

    log_step("TASK_4", "FETCH", "Retrieving the full transaction history from the CSV logs.")
    txns = pd.read_parquet(os.path.join(PATHS["transactions"], "transactions_merged.parquet"))


    def get_insight(df, description, name):
        log_step("TASK_4", "AUDIT", f"Analyzing the {name} dataset for missing values and schema health.")
        completeness = int((1 - df.isna().sum().sum()/df.size)*100)
        
        attr_samples = []
        for col in df.columns[:4]:
            samples = df[col].dropna().unique()[:3].tolist()
            attr_samples.append(f"{col}: (e.g., {', '.join(map(str, samples))})")
            
        return {
            "desc": description, "rows": len(df), "cols": len(df.columns),
            "score": completeness, "samples": attr_samples
        }

    ds_info = {
        "User Profiles": get_insight(users_raw, "Contains customer demographics like Age and Location.", "Users"),
        "Product Catalog": get_insight(prods, "List of items sold, including prices and stock status.", "Products"),
        "Transaction Logs": get_insight(txns, "History of user purchases and product ratings.", "Transactions")
    }

    log_step("TASK_4", "REPORT", "Generating the visual Data Quality Report.")
    c = canvas.Canvas(REPORT_PATH, pagesize=A4)
    w, h = A4
    c.setFillColorRGB(0.07, 0.15, 0.3); c.rect(0, h-70, w, 70, fill=1)
    c.setFillColor(colors.white); c.setFont("Helvetica-Bold", 20)
    c.drawString(50, h-35, "RecoMart: Data Quality Report")
    c.setFont("Helvetica", 10); c.drawString(50, h-55, f"Analysis of Raw Files Ingested on: {DATE_STR}")

    y = h - 110
    for name, data in ds_info.items():
        c.setFillColor(colors.black); c.setFont("Helvetica-Bold", 14); c.drawString(50, y, f"Dataset: {name}")
        score_color = colors.darkgreen if data['score'] >= 95 else colors.darkorange
        c.setFillColor(score_color); c.setFont("Helvetica-Bold", 11); c.drawRightString(530, y, f"Health Score: {data['score']}% Complete")
        y -= 15; c.setFillColor(colors.black); c.setFont("Helvetica-Oblique", 10); c.drawString(65, y, data['desc'])
        y -= 20; c.setFont("Helvetica", 10); c.drawString(80, y, f"• Volume: {data['rows']} entries across {data['cols']} attributes.")
        y -= 15; c.drawString(80, y, "• Key Attributes Identified:")
        y -= 15; c.setFont("Helvetica", 9); c.setFillColor(colors.grey)
        for s in data['samples']:
            c.drawString(100, y, f"- {s}"); y -= 12
        y -= 10; c.setStrokeColor(colors.lightgrey); c.line(50, y, 540, y); y -= 30

    c.setFillColor(colors.black); c.setFont("Helvetica-Bold", 13); c.drawString(50, y, "Critical Quality Findings:")
    y -= 25; c.setFont("Helvetica", 10); c.drawString(70, y, "■ No major data anomalies detected. Data is safe for preparation.")
    c.save()
    log_step("TASK_4", "SUCCESS", f"The Data Quality Report is ready : {REPORT_PATH}")

except Exception as e:
    log_step("TASK_4", "ERROR", f"The Data Quality report was interrupted by an error: {str(e)}")

[2026-01-27 18:22:31] [TASK_4] [START] Initiating data profiling and validation for data quality.
[2026-01-27 18:22:31] [TASK_4] [FETCH] Reading the first batch of user records from users1.csv.
[2026-01-27 18:22:31] [TASK_4] [FETCH] Opening the products.json file to extract item metadata.
[2026-01-27 18:22:31] [TASK_4] [FETCH] Retrieving the full transaction history from the CSV logs.
[2026-01-27 18:22:31] [TASK_4] [AUDIT] Analyzing the Users dataset for missing values and schema health.
[2026-01-27 18:22:31] [TASK_4] [AUDIT] Analyzing the Products dataset for missing values and schema health.
[2026-01-27 18:22:31] [TASK_4] [AUDIT] Analyzing the Transactions dataset for missing values and schema health.
[2026-01-27 18:22:31] [TASK_4] [REPORT] Generating the visual Data Quality Report.
[2026-01-27 18:22:31] [TASK_4] [SUCCESS] The Data Quality Report is ready : Data_Quality_Report.pdf


In [16]:
dt.datetime.now().strftime("%Y-%m-%d")

'2026-01-27'